# News Project
> The goal of this project is to create a newspaper like app for dialy news. 

I want a facts first, unbiased version of the news. I dont want to endlessly scroll news and memes all day. 

In [6]:
import feedparser
import time
from datetime import datetime

def fetch_google_news(country='US', language='en', topic=None, query=None):
    base_url = f"https://news.google.com/rss?hl={language}&gl={country}&ceid={country}:{language}"
    
    if topic:
        base_url += f"&topic={topic}"
    elif query:
        base_url = f"https://news.google.com/rss/search?q={query}&hl={language}&gl={country}&ceid={country}:{language}"

    print(f"Fetching news from: {base_url}")
    
    feed = feedparser.parse(base_url)
    
    articles = []
    for entry in feed.entries:
        articles.append({
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'summary': entry.summary,
            'source': entry.source.href if hasattr(entry, 'source') else 'Unknown'
        })
    
    return articles


# Example usage
LANGUAGE_CODE = 'en'
COUNTRY_CODE = 'US'

# us_news = fetch_google_news(country=COUNTRY_CODE, language=LANGUAGE_CODE)
world_news = fetch_google_news(language=LANGUAGE_CODE, topic='world')
# print count of articles
print(f"Number of world news articles fetched: {len(world_news)}")

for article in world_news:
    print(f"Title: {article['title']}")
    print(f"Link: {article['link']}")
    print(f"Published: {article['published']}")
    print(f"Summary: {article['summary']}")
    print(f"Source: {article['source']}\n")
    



Fetching news from: https://news.google.com/rss?hl=en&gl=US&ceid=US:en&topic=world
Number of world news articles fetched: 37
Title: DNI Gabbard, in Partnership with DOJ, FBI, CIA, and NARA, Release Martin Luther King Jr. Assassination Files - ODNI.gov
Link: https://news.google.com/rss/articles/CBMikAFBVV95cUxOMXFkMTljbzRrV1o2Tlh6RE9rTjdUTzZoV1FMYVdtUTkzRnRvTWhMb0lIRzUwWTFHRkpKY2FMcWw0VmN5bDd1RW1qQm9PbnhQaDlPQ1ktdDlZeERTbEdfdTVLRmR5dUNZRHp3V3FTTGd1WDNXOG9DSG1BWENOdC14UWlHRzE0d3lLLWxhaEZrREY?oc=5
Published: Mon, 21 Jul 2025 19:41:02 GMT
Summary: <ol><li><a href="https://news.google.com/rss/articles/CBMikAFBVV95cUxOMXFkMTljbzRrV1o2Tlh6RE9rTjdUTzZoV1FMYVdtUTkzRnRvTWhMb0lIRzUwWTFHRkpKY2FMcWw0VmN5bDd1RW1qQm9PbnhQaDlPQ1ktdDlZeERTbEdfdTVLRmR5dUNZRHp3V3FTTGd1WDNXOG9DSG1BWENOdC14UWlHRzE0d3lLLWxhaEZrREY?oc=5" target="_blank">DNI Gabbard, in Partnership with DOJ, FBI, CIA, and NARA, Release Martin Luther King Jr. Assassination Files</a>&nbsp;&nbsp;<font color="#6f6f6f">ODNI.gov</font></li><li><a h

In [16]:
from bs4 import BeautifulSoup
import requests
import json

session = requests.Session()

# List to holde articles [(url, content), ...]
articles = []

def get_real_url(google_news_url):
    try:
        # Step 1: Get the Google News page
        resp = requests.get(google_news_url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # Step 2: Extract the special data
        cwiz = soup.select_one('c-wiz[data-p]')
        if not cwiz:
            return None
            
        data = cwiz.get('data-p')
        obj = json.loads(data.replace('%.@.', '["garturlreq",'))
        
        # Step 3: Make POST request to Google's API
        payload = {
            'f.req': json.dumps([[['Fbv4je', json.dumps(obj[:-6] + obj[-2:]), 'null', 'generic']]])
        }
        
        headers = {
            'content-type': 'application/x-www-form-urlencoded;charset=UTF-8',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.post("https://news.google.com/_/DotsSplashUi/data/batchexecute", 
                               headers=headers, data=payload)
        
        # Step 4: Parse the response to get the real URL
        array_string = json.loads(response.text.replace(")]}'", ""))[0][2]
        article_url = json.loads(array_string)[1]
        
        return article_url
        
    except Exception as e:
        print(f"Error getting real URL: {e}")
        return None

def scrape_article(google_news_link):
    try:
        # Scrape
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

        # redirect_response = requests.get(google_link, headers=headers, allow_redirects=True, timeout=30)
        real_news_url = get_real_url(google_news_link)
        print(f"Redirected URL: {real_news_url}")
        response = requests.get(real_news_url, headers=headers, allow_redirects=True, timeout=30) 
        time.sleep(2)
        soup = BeautifulSoup(response.content, 'html.parser')
        article = (soup.find('article') or 
                  soup.find('div', class_='article-content') or
                #   soup.find('div', class_='FITT_Article_main__body') or
                #   soup.find('div', class_='theme-e') or
                  soup.find('main'))
        
        # If no article try bypassing
        if not article:
            # bypass_link = f"https://archive.is/latest/{real_news_url}"
            bypass_link = f"https://web.archive.org/web/{real_news_url}"
            print(f"Bypassing with: {bypass_link}")
            response = requests.get(bypass_link, headers=headers, allow_redirects=False, timeout=30)
            if response.status_code in [301, 302, 303, 307, 308]:
                redirect_url = response.headers.get('Location')
                if redirect_url and 'web.archive' in redirect_url:
                    time.sleep(20)
                    print(f"Following archive.is redirect to: {redirect_url}")
                    response = session.get(redirect_url, headers=headers, timeout=30)
                else:
                    time.sleep(10)
                    print(f"Blocked redirect to: {redirect_url}")
            print(f"Response status code: {response.status_code}")
            print(f"Response URL: {response.url}")
            time.sleep(2)
            soup = BeautifulSoup(response.content, 'html.parser')
            article = (soup.find('article') or 
                  soup.find('div', class_='article-content') or
                #   soup.find('div', class_='FITT_Article_main__body') or
                #   soup.find('div', class_='theme-e') or
                  soup.find('main'))
                  

        return article
    except Exception as e:
        print(f"Error scraping {google_news_link}: {e}")
        return ""


# Scrape from sites to get articles
for article in world_news:
    full_content = scrape_article(article['link'])
    # Extract text from BeautifulSoup Tag before slicing
    if hasattr(full_content, 'get_text'):
        content_text = full_content.get_text()
    else:
        content_text = str(full_content)
    articles.append((article['link'], content_text))  # First 100 characters for testing
    # print(f"Full content: {content_text}\n")
    print(f"Article Done")


Redirected URL: https://www.dni.gov/index.php/newsroom/press-releases/press-releases-2025/4087-pr-16-25
Bypassing with: https://web.archive.org/web/https://www.dni.gov/index.php/newsroom/press-releases/press-releases-2025/4087-pr-16-25
Following archive.is redirect to: https://web.archive.org/web/20250722002709/https://www.dni.gov/index.php/newsroom/press-releases/press-releases-2025/4087-pr-16-25
Response status code: 200
Response URL: https://web.archive.org/web/20250722002709/https://www.dni.gov/index.php/newsroom/press-releases/press-releases-2025/4087-pr-16-25
Article Done
Redirected URL: https://www.npr.org/2025/07/21/nx-s1-5462675/harvard-trump-court-hearing-boston
Article Done
Redirected URL: https://apnews.com/article/europe-israel-hamas-war-gaza-e4062cffa9585790061105236a93d8e5
Article Done
Redirected URL: https://www.nbcnews.com/politics/donald-trump/trumps-big-beautiful-bill-add-3-trillion-debt-cbo-says-rcna220004
Article Done
Redirected URL: https://www.npr.org/2025/07/21/

In [12]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get individual database components
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

# Construct the database URL
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Database connection utilities
import psycopg2

def read_from_db(query):
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        print(f"Error reading from database: {e}")
        return []

def execute_db_query(query):
    """Execute queries that don't return data (CREATE, INSERT, UPDATE, DELETE)"""
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
        cursor.close()
        conn.close()
        print("Query executed successfully")
        return True
    except Exception as e:
        print(f"Error executing query: {e}")
        return False

# Create a table for articles
# Columns: id, title, raw_content, link
def create_article_table():
    """Create the articles table with specified columns"""
    
    # Drop table if exists (optional - remove if you don't want to overwrite)
    drop_query = "DROP TABLE IF EXISTS articles;"
    
    # Create table query
    create_query = """
    CREATE TABLE articles (
        id SERIAL PRIMARY KEY,
        title VARCHAR(500) NOT NULL,
        raw_content TEXT,
        link VARCHAR(1000) UNIQUE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    
    # Create index for better performance on common searches
    index_query = """
    CREATE INDEX IF NOT EXISTS idx_articles_title ON articles(title);
    CREATE INDEX IF NOT EXISTS idx_articles_link ON articles(link);
    """
    
    print("Creating articles table...")
    
    # Execute drop (optional)
    execute_db_query(drop_query)
    
    # Execute create
    if execute_db_query(create_query):
        print("Articles table created successfully!")
        
        # Create indexes
        execute_db_query(index_query)
        print("Indexes created successfully!")
        
        # Verify table creation
        verify_query = "SELECT table_name FROM information_schema.tables WHERE table_name='articles';"
        result = read_from_db(verify_query)
        if result:
            print("Table 'articles' confirmed in database")
        else:
            print("Warning: Could not verify table creation")
    else:
        print("Failed to create articles table")

def show_table_structure():
    """Display the structure of the articles table"""
    query = """
    SELECT column_name, data_type, character_maximum_length, is_nullable, column_default
    FROM information_schema.columns 
    WHERE table_name = 'articles'
    ORDER BY ordinal_position;
    """
    
    results = read_from_db(query)
    if results:
        print("\nArticles table structure:")
        print("Column Name | Data Type | Max Length | Nullable | Default")
        print("-" * 60)
        for row in results:
            col_name, data_type, max_length, nullable, default = row
            max_length = max_length if max_length else "N/A"
            default = default if default else "None"
            print(f"{col_name:<12} | {data_type:<10} | {max_length:<10} | {nullable:<8} | {default}")
    else:
        print("Could not retrieve table structure")

def list_all_tables():
    """List all tables in the database"""
    query = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
    table_names = read_from_db(query)
    print("Table names in the database:")
    for table in table_names:
        print(f"- {table[0]}")

# Sample functions for working with the articles table
def insert_article(title, raw_content, link):
    """Insert a new article into the table"""
    query = """
    INSERT INTO articles (title, raw_content, link) 
    VALUES (%s, %s, %s) RETURNING id;
    """
    
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()
        cursor.execute(query, (title, raw_content, link))
        article_id = cursor.fetchone()[0]
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Article inserted successfully with ID: {article_id}")
        return article_id
    except Exception as e:
        print(f"Error inserting article: {e}")
        return None

def get_all_articles():
    """Retrieve all articles from the table"""
    query = "SELECT id, title, link, created_at FROM articles ORDER BY created_at DESC;"
    return read_from_db(query)

# Create the articles table
create_article_table()
# Show the table structure
show_table_structure()
# List all tables to confirm
list_all_tables()

Creating articles table...
Query executed successfully
Query executed successfully
Articles table created successfully!
Query executed successfully
Indexes created successfully!
Table 'articles' confirmed in database

Articles table structure:
Column Name | Data Type | Max Length | Nullable | Default
------------------------------------------------------------
id           | integer    | N/A        | NO       | nextval('articles_id_seq'::regclass)
title        | character varying | 500        | NO       | None
raw_content  | text       | N/A        | YES      | None
link         | character varying | 1000       | YES      | None
created_at   | timestamp without time zone | N/A        | YES      | CURRENT_TIMESTAMP
updated_at   | timestamp without time zone | N/A        | YES      | CURRENT_TIMESTAMP
Table names in the database:
- articles


In [17]:
# For articles Put each in the db
# Insert articles into database
# Just inserting link and raw content for now
# articles = [(url, content), ...]

import psycopg2
import re

def insert_articles_batch():
    """Insert all articles from the articles list into the database"""
    
    if not articles:
        print("No articles found to insert")
        return
    
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()
        
        successful_inserts = 0
        failed_inserts = 0
        
        for i, (url, content) in enumerate(articles):
            try:
                # Clean content - remove excessive whitespace
                cleaned_content = re.sub(r'\n\s*\n', '\n\n', content.strip())
                cleaned_content = re.sub(r' +', ' ', cleaned_content)
                
                # Insert into database - just link and content for now
                insert_query = """
                INSERT INTO articles (title, raw_content, link) 
                VALUES (%s, %s, %s) 
                ON CONFLICT (link) DO UPDATE SET
                    raw_content = EXCLUDED.raw_content,
                    updated_at = CURRENT_TIMESTAMP
                RETURNING id;
                """
                
                # Use a placeholder title for now
                placeholder_title = f"Article {i+1}"
                
                cursor.execute(insert_query, (placeholder_title, cleaned_content, url))
                article_id = cursor.fetchone()[0]
                
                print(f"✓ Article {i+1}/{len(articles)} inserted successfully (ID: {article_id})")
                print(f"  URL: {url}")
                print(f"  Content length: {len(cleaned_content)} characters")
                print()
                
                successful_inserts += 1
                
            except Exception as e:
                print(f"✗ Error inserting article {i+1}: {str(e)}")
                print(f"  URL: {url}")
                print()
                failed_inserts += 1
                continue
        
        # Commit all successful insertions
        conn.commit()
        cursor.close()
        conn.close()
        
        print(f"\n=== INSERTION SUMMARY ===")
        print(f"Total articles processed: {len(articles)}")
        print(f"Successfully inserted: {successful_inserts}")
        print(f"Failed insertions: {failed_inserts}")
        print(f"Success rate: {(successful_inserts/len(articles)*100):.1f}%")
        
    except Exception as e:
        print(f"Database connection error: {e}")

def verify_insertions():
    """Verify that articles were inserted correctly"""
    try:
        query = "SELECT COUNT(*), AVG(LENGTH(raw_content)) FROM articles;"
        result = read_from_db(query)
        
        if result:
            count, avg_length = result[0]
            print(f"\nDatabase verification:")
            print(f"Total articles in database: {count}")
            print(f"Average content length: {avg_length:.0f} characters")
            
        # Show recent articles
        recent_query = """
        SELECT id, title, LENGTH(raw_content) as content_length, created_at 
        FROM articles 
        ORDER BY created_at DESC 
        LIMIT 5;
        """
        recent_articles = read_from_db(recent_query)
        
        if recent_articles:
            print(f"\nRecent articles:")
            for article in recent_articles:
                article_id, title, length, created = article
                print(f"  ID {article_id}: {title[:60]}{'...' if len(title) > 60 else ''} ({length} chars)")
                
    except Exception as e:
        print(f"Error verifying insertions: {e}")

# Run the insertion
print("Starting article insertion...")
print(f"Found {len(articles)} articles to process")
print()

insert_articles_batch()
verify_insertions()

Starting article insertion...
Found 37 articles to process

✓ Article 1/37 inserted successfully (ID: 1)
  URL: https://news.google.com/rss/articles/CBMikAFBVV95cUxOMXFkMTljbzRrV1o2Tlh6RE9rTjdUTzZoV1FMYVdtUTkzRnRvTWhMb0lIRzUwWTFHRkpKY2FMcWw0VmN5bDd1RW1qQm9PbnhQaDlPQ1ktdDlZeERTbEdfdTVLRmR5dUNZRHp3V3FTTGd1WDNXOG9DSG1BWENOdC14UWlHRzE0d3lLLWxhaEZrREY?oc=5
  Content length: 4 characters

✓ Article 2/37 inserted successfully (ID: 2)
  URL: https://news.google.com/rss/articles/CBMihgFBVV95cUxQcXJSVy1Ld2lXYm9wMWkxYnhKYjNTeG1sWHMxc0lwWnpVODBGdUVVU0pqUlZSeGlCdndhZGFfcWFXYzVHTDhla1BUcGNNcUtJVmJvdlF0TFNfU3Q2cjVCMk5OUmhINWswM2dBcG9uci11bS1JUS1VaktPRVcxcFhQY1F5OUtlZw?oc=5
  Content length: 9970 characters

✓ Article 3/37 inserted successfully (ID: 3)
  URL: https://news.google.com/rss/articles/CBMikgFBVV95cUxNUFVsYkRkbGw3Q0V3ZUJIUkVpZlZMUGppb0lpRXQ4Q0o3Um9Ya1pDcXJsc3UxM21QN1RabDdmd0tpbHVMV3FHVVNZMEFGLTU2MXkxMUdrV1RrR040Y1BwU1pQeWtEM182aG5VNjdLQlVaaG5kNVk4MnZXS1hmYlo0MEdkN1pUeGRoQV9NVkdwZnktdw?oc=5
 